In [607]:
import pandas as pd

In [608]:
df = pd.read_csv('f1_2018.csv')

In [609]:
df.head()

,Unnamed: 0,raceId,year,circuitId,name,date,driverId,constructorId,laps,time,statusId,driver_name,Temperature (C),Humidity,WindSpeed,TrackTemp,Rain
0,0,989,2018,1,Australian Grand Prix,2018-03-25,20,6,58,1:29:33.283,1,vettel,24.077477,30.915315,3.691892,36.324324,True
1,1,990,2018,3,Bahrain Grand Prix,2018-04-08,20,6,57,1:32:01.940,1,vettel,27.982524,47.363107,0.958252,32.198058,False
2,2,991,2018,17,Chinese Grand Prix,2018-04-15,817,9,56,1:35:36.380,1,ricciardo,19.446429,24.089286,1.837500,37.019643,True
3,3,992,2018,73,Azerbaijan Grand Prix,2018-04-29,1,131,51,1:43:44.291,1,hamilton,16.661404,45.651754,2.222807,25.251754,False
4,4,993,2018,4,Spanish Grand Prix,2018-05-13,1,131,66,1:35:29.972,1,hamilton,16.050476,52.286667,1.952381,32.339048,True


In [610]:
def heat_index(t,h):
    coef = -42.379 + (2.04901523 * t)+(10.14333127 * h)-(0.22475541 * t * h)-(6.83783e-3 * t**2)-(5.481717e-2 * h**2 ) + (1.22874e-3 * t**2 * h) + (8.5282e-4 * t * h**2) - (1.99e-6 * t**2 * h**2)
    if coef >= 80 and coef <=90:
        return "Caution"
    elif coef >= 91 and coef <= 103:
        return "Extreme Caution"
    elif coef >= 104 and coef <= 124:
        return "Danger"
    elif coef >= 125:
        return "Extreme Danger" 
    else:
        return "Fine"
   
df["Temperature (F)"] = (df["Temperature (C)"] * 9/5) + 32
df["Heat Index"] = ''
for i in range(df["Temperature (F)"].shape[0]):
    temp = df["Temperature (F)"].loc[i]
    humidity = df["Humidity"].loc[i]
    df["Heat Index"].loc[i] = heat_index(temp, humidity)

/var/folders/wm/j6zh4bsx5kd608f73_8dxtn40000gn/T/ipykernel_25219/2863760274.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Heat Index"].loc[i] = heat_index(temp, humidity)


In [611]:
df["Heat Index"].value_counts()

Fine               75
Caution            31
Extreme Caution     8
Name: Heat Index, dtype: int64

In [612]:
#categorizing wind speed (units of m/s)
df["WindSpeed"] = df["WindSpeed"] * 2.2369
df["WindSpeed (mph)"] = df["WindSpeed"]
df["WindSpeed (mph)"].describe()

count    114.000000
mean       3.672336
std        2.255937
min        0.607324
25%        2.249856
50%        3.357038
75%        4.560313
max       12.274002
Name: WindSpeed (mph), dtype: float64

In [613]:
def wind_speed(x):
    if x >= 1 and x < 4:
        return "Light Air"
    elif x >= 4 and x < 8:
        return "Light Breeze"
    elif x >= 8 and x < 13:
        return "Gentle Breeze"
    else:
        return "Calm"

df["Wind"] = df["WindSpeed (mph)"].apply(lambda x: wind_speed(x))



In [614]:
df["Wind"].value_counts()

Light Air        53
Light Breeze     41
Calm             14
Gentle Breeze     6
Name: Wind, dtype: int64

In [615]:
df.columns

Index(['Unnamed: 0', 'raceId', 'year', 'circuitId', 'name', 'date', 'driverId',
       'constructorId', 'laps', 'time', 'statusId', 'driver_name',
       'Temperature (C)', 'Humidity', 'WindSpeed', 'TrackTemp', 'Rain',
       'Temperature (F)', 'Heat Index', 'WindSpeed (mph)', 'Wind'],
      dtype='object')

In [616]:
df = df.drop(["Temperature (C)", "Temperature (F)", "WindSpeed", "WindSpeed (mph)", "Humidity"], axis=1)

In [617]:
df.head()

,Unnamed: 0,raceId,year,circuitId,name,date,driverId,constructorId,laps,time,statusId,driver_name,TrackTemp,Rain,Heat Index,Wind
0,0,989,2018,1,Australian Grand Prix,2018-03-25,20,6,58,1:29:33.283,1,vettel,36.324324,True,Fine,Gentle Breeze
1,1,990,2018,3,Bahrain Grand Prix,2018-04-08,20,6,57,1:32:01.940,1,vettel,32.198058,False,Caution,Light Air
2,2,991,2018,17,Chinese Grand Prix,2018-04-15,817,9,56,1:35:36.380,1,ricciardo,37.019643,True,Fine,Light Breeze
3,3,992,2018,73,Azerbaijan Grand Prix,2018-04-29,1,131,51,1:43:44.291,1,hamilton,25.251754,False,Caution,Light Breeze
4,4,993,2018,4,Spanish Grand Prix,2018-05-13,1,131,66,1:35:29.972,1,hamilton,32.339048,True,Caution,Light Breeze


In [618]:
df["TrackTemp"].describe()

count    114.000000
mean      35.190365
std        9.108954
min       15.205755
25%       29.630616
50%       34.321192
75%       41.207850
max       54.525620
Name: TrackTemp, dtype: float64

In [619]:
def track_cond(x):
    if x <= 20:
        return "Cold"
    elif x > 20 and x <= 40:
        return "Ideal"
    if x > 40:
        return "Hot"
df["TrackTemp"] = df["TrackTemp"].apply(lambda x: track_cond(x))

In [620]:
#changing constructorId to constructor name
constructor = pd.read_csv("constructors.csv")
constructor = constructor[["constructorId", "name"]]
constructor = constructor.set_index('constructorId').to_dict()["name"]
df["constructorId"] = df["constructorId"].apply(lambda x: constructor[x])


In [621]:
pitstops = pd.read_csv("pit_stops.csv")
pitstops.shape


(10052, 7)

In [622]:
pitstops = pitstops[pitstops["raceId"].isin(df['raceId'])].reset_index(drop=True)

In [623]:
pitstops.shape

(3801, 7)

In [624]:
pitstop_time = pitstops.groupby(['raceId', 'driverId'])['milliseconds'].sum()
pitstop_count = pitstops.groupby(['raceId', 'driverId'])['stop'].max()
pitstop_time = pd.DataFrame(pitstop_time)
pitstop_count = pd.DataFrame(pitstop_count)
df["pitstop time"] = ''
df['# of pitstops'] = ''

In [625]:

for i in range(df.shape[0]):
    race = df['raceId'].loc[i]
    winner = df['driverId'].loc[i]
    try:
        df['pitstop time'].loc[i] = pitstop_time.loc[race, winner]["milliseconds"]
        df['# of pitstops'].loc[i] = pitstop_count.loc[race, winner]['stop']
    except KeyError as err:
        df["pitstop time"].loc[i] = pd.NA
        df['# of pitstops'].loc[i] = pd.NA
df['pitstop time'] = df['pitstop time'] / 1000


/var/folders/wm/j6zh4bsx5kd608f73_8dxtn40000gn/T/ipykernel_25219/3870332311.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pitstop time'].loc[i] = pitstop_time.loc[race, winner]["milliseconds"]
/var/folders/wm/j6zh4bsx5kd608f73_8dxtn40000gn/T/ipykernel_25219/3870332311.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['# of pitstops'].loc[i] = pitstop_count.loc[race, winner]['stop']
/var/folders/wm/j6zh4bsx5kd608f73_8dxtn40000gn/T/ipykernel_25219/3870332311.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [626]:
df.head()

,Unnamed: 0,raceId,year,circuitId,name,date,driverId,constructorId,laps,time,statusId,driver_name,TrackTemp,Rain,Heat Index,Wind,pitstop time,# of pitstops
0,0,989,2018,1,Australian Grand Prix,2018-03-25,20,Ferrari,58,1:29:33.283,1,vettel,Ideal,True,Fine,Gentle Breeze,21.787,1
1,1,990,2018,3,Bahrain Grand Prix,2018-04-08,20,Ferrari,57,1:32:01.940,1,vettel,Ideal,False,Caution,Light Air,24.524,1
2,2,991,2018,17,Chinese Grand Prix,2018-04-15,817,Red Bull,56,1:35:36.380,1,ricciardo,Ideal,True,Fine,Light Breeze,45.733,2
3,3,992,2018,73,Azerbaijan Grand Prix,2018-04-29,1,Mercedes,51,1:43:44.291,1,hamilton,Ideal,False,Caution,Light Breeze,41.706,2
4,4,993,2018,4,Spanish Grand Prix,2018-05-13,1,Mercedes,66,1:35:29.972,1,hamilton,Ideal,True,Caution,Light Breeze,22.085,1


In [627]:
#dropped_col = ['Unnamed: 0', 'raceId', 'laps', 'statusId', 'driverId', 'milliseconds']
#df = df.drop(dropped_col, axis=1)

In [628]:
df.isna().sum()

Unnamed: 0       0
raceId           0
year             0
circuitId        0
name             0
date             0
driverId         0
constructorId    0
laps             0
time             0
statusId         0
driver_name      0
TrackTemp        0
Rain             0
Heat Index       0
Wind             0
pitstop time     1
# of pitstops    1
dtype: int64

In [629]:
df[df.isnull().any(axis=1)]

,Unnamed: 0,raceId,year,circuitId,name,date,driverId,constructorId,laps,time,statusId,driver_name,TrackTemp,Rain,Heat Index,Wind,pitstop time,# of pitstops
70,70,1063,2021,13,Belgian Grand Prix,2021-08-29,830,Red Bull,1,3:27.071,1,max_verstappen,Cold,True,Fine,Light Air,<NA>,<NA>


In [630]:
df['# of pitstops'].loc[70] = 2
df['pitstop time'].loc[70] = 45.204

/var/folders/wm/j6zh4bsx5kd608f73_8dxtn40000gn/T/ipykernel_25219/192166158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['# of pitstops'].loc[70] = 2
/var/folders/wm/j6zh4bsx5kd608f73_8dxtn40000gn/T/ipykernel_25219/192166158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pitstop time'].loc[70] = 45.204


In [631]:
df_qual = pd.read_csv('qualifying.csv')
df_qual = df_qual[["raceId", "driverId", "position"]]
df_qual = df_qual.rename(columns={"position": "qualifying_position"})
df = df.merge(df_qual, how="inner", on=["raceId", "driverId"])
df.head()

,Unnamed: 0,raceId,year,circuitId,name,date,driverId,constructorId,laps,time,statusId,driver_name,TrackTemp,Rain,Heat Index,Wind,pitstop time,# of pitstops,qualifying_position
0,0,989,2018,1,Australian Grand Prix,2018-03-25,20,Ferrari,58,1:29:33.283,1,vettel,Ideal,True,Fine,Gentle Breeze,21.787,1,3
1,1,990,2018,3,Bahrain Grand Prix,2018-04-08,20,Ferrari,57,1:32:01.940,1,vettel,Ideal,False,Caution,Light Air,24.524,1,1
2,2,991,2018,17,Chinese Grand Prix,2018-04-15,817,Red Bull,56,1:35:36.380,1,ricciardo,Ideal,True,Fine,Light Breeze,45.733,2,6
3,3,992,2018,73,Azerbaijan Grand Prix,2018-04-29,1,Mercedes,51,1:43:44.291,1,hamilton,Ideal,False,Caution,Light Breeze,41.706,2,2
4,4,993,2018,4,Spanish Grand Prix,2018-05-13,1,Mercedes,66,1:35:29.972,1,hamilton,Ideal,True,Caution,Light Breeze,22.085,1,1


In [ ]:
import seaborn as sns
sns.countplot()